In [1]:

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score
# thực hiện các chức năng đánh giá lỗi dự đoán cho các mục đích cụ thể. Các chỉ số này được trình bày chi tiết trong các phần về
# chỉ số Phân loại , chỉ số xếp hạng Đa nhãn , chỉ số hồi quy và chỉ số Nhóm .
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [4]:
forest_fires = pd.read_csv("../input/forestfire69/forestfires.csv")
forest_fires.head(10)

<b>Gioi thieu ve bo du lieu</b>

In [5]:
# Số hàng và cột:
forest_fires.shape

In [7]:
# Xem các thuoc tinh :
forest_fires.columns

In [8]:
#Xem thông tin của các thuoc tính
forest_fires.info()

In [11]:
#Xem thông tin của các thuoc tính
forest_fires.describe(include='object')

<b>Thông tin chi tiết</b>

In [13]:
# Các đám cháy rừng được phân bố như thế nào trong vườn
import seaborn as sns
sns.distplot(forest_fires["area"])

In [15]:
print(forest_fires["area"].describe().T)
plt.boxplot(forest_fires["area"])

In [17]:
# Dien tich khong bị chay
print(len(forest_fires[forest_fires["area"]==0.0])*100/forest_fires.shape[0])

<u> 47,77% - khu vực không bị cháy và khoảng 53% - khu vực bị cháy</u>

.


In [19]:
# Giá trị khu vực bị ảnh hưởng cao nhất, thấp nhất và trung bình:
print(forest_fires["area"].agg(["max","min","mean"]))

In [27]:
#Tỉ le khu vực tb bị ảnh huong
len(forest_fires[forest_fires["area"]>=12.84])*100/forest_fires.shape[0]

In [23]:
#Tỉ le khu vực cao nhat bị ảnh huong
len(forest_fires[forest_fires["area"]==1090.84])*100/forest_fires.shape[0]

In [28]:
# Vị trí của khu vực bị ảnh hưởng nặng
forest_fires[forest_fires["area"]== 1090.840000][["X","Y"]]

In [37]:
# Giá trị nhiệt độ, mưa, gió, RH và Độ ẩm (FFMC, DMC, DC) tương ứng với khu vực bị cháy cao
x=forest_fires[forest_fires["area"]==1090.84]
x.iloc[:,0:12]

In [38]:
# Tháng dễ xảy ra cháy rừng nhất
print(forest_fires["month"].value_counts())

In [66]:
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.hist(forest_fires["month"],bins=20)
plt.xlabel(" months ")
plt.ylabel("frequency")

<b>Từ biểu đồ trên tháng 8,9 có thể xảy ra cao nhất và th4,th5 ít bị nhất</b>

In [70]:
# Tỉ lệkhu vực bị cháy trong tháng 8
forest_fires2=forest_fires[(forest_fires["month"]=="aug")&(forest_fires["area"]>0)]
len(forest_fires2[forest_fires2["month"]=="aug"])*100/forest_fires.shape[0]


<b>Giá trị tối đa, tối thiểu và trung bình của tất cả các yếu tố có thể ảnh hưởng đến cháy rừng vào tháng 8 </b>

In [71]:
forest_fires2["area"].agg(["max","min","mean"])

In [72]:
# So sánh giá trị các yếu tố của tất cả các tháng voi tháng 8
forest_fires2[forest_fires2["month"]=="aug"][["temp","rain","wind","RH","FFMC","DMC","DC"]].agg(["max","min","mean"]).T

In [75]:
print(forest_fires["day"].value_counts())
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
plt.hist(forest_fires["day"],bins=17)
plt.xlabel(" days  ")
plt.ylabel("frequency")

In [76]:
forest_fires.groupby(["month"])["day"].value_counts()

<b>Nhiệt độ cao nhất được ghi lại trong tháng nào?
Gió cực đại được ghi nhận vào tháng nào?
Mưa nhiều nhất được ghi nhận vào tháng nào?
RH tối đa được ghi vào tháng nào?</b>

In [78]:
plt.figure(figsize=(10,6))
plt.subplot(2,2,1)
plt.bar(x=forest_fires["month"],height=forest_fires["temp"])
plt.xlabel(" month  ")
plt.ylabel("temp in celcius")
plt.subplot(2,2,2)
plt.bar(x=forest_fires["month"],height=forest_fires["wind"])
plt.xlabel(" month  ")
plt.ylabel("wind speed ")
plt.subplot(2,2,3)
plt.bar(x=forest_fires["month"],height=forest_fires["RH"])
plt.xlabel(" month ")
plt.ylabel("Relative humidity in %")
plt.subplot(2,2,4)
plt.bar(x=forest_fires["month"],height=forest_fires["rain"])
plt.xlabel(" month ")
plt.ylabel("rain in mm")

In [79]:
print(forest_fires["temp"].max())
print(forest_fires["wind"].max())
print(forest_fires["rain"].max())
print(forest_fires["RH"].max())

<b>FFMC bị ảnh hưởng như thế nào bởi Nhiệt độ, mưa, gió và RH </b>

In [81]:
plt.figure(figsize=(10,6))
plt.subplot(2,2,1)
plt.scatter(x=forest_fires["temp"],y=forest_fires["FFMC"])
plt.xlabel(" temp in celcius  ")
plt.ylabel("FFMC rating value")
plt.subplot(2,2,2)
plt.scatter(x=forest_fires["wind"],y=forest_fires["FFMC"])
plt.xlabel(" wind speed  ")
plt.ylabel("FFMC rating value ")
plt.subplot(2,2,3)
plt.scatter(x=forest_fires["RH"],y=forest_fires["FFMC"])
plt.xlabel(" Relative humidity  ")
plt.ylabel("FFMC rating value")
plt.subplot(2,2,4)
plt.scatter(x=forest_fires["rain"],y=forest_fires["FFMC"])
plt.xlabel(" rain  value in mm  ")
plt.ylabel("FFMC rating value")

Nếu nhiệt độ khoảng 15-20 và giá trị FFMC khoảng 80-90 thì đó là điều kiện cực kỳ có khả năng xảy ra cháy rừng có thể gây thiệt hại cho khu vực

Nếu không có mưa thì hầu hết giá trị FFMC sẽ vào khoảng 80-90

Nếu RH khoảng 40-60 thì giá trị FFMC ở khoảng 80-90

Nếu gió khoảng 3-4 km / h và giá trị FFMC lớn hơn ở 80-90

<b>DMC bị ảnh hưởng như thế nào bởi Nhiệt độ, mưa, gió và RH </b>


In [82]:
plt.figure(figsize=(10,6))
plt.subplot(2,2,1)
plt.scatter(x=forest_fires["DMC"],y=forest_fires["temp"])
plt.xlabel(" DMC rating value  ")
plt.ylabel("temp in celcius")
plt.subplot(2,2,2)
plt.scatter(x=forest_fires["DMC"],y=forest_fires["wind"])
plt.xlabel(" DMC rating value  ")
plt.ylabel("wind speed ")
plt.subplot(2,2,3)
plt.scatter(x=forest_fires["DMC"],y=forest_fires["RH"])
plt.xlabel(" DMC rating value  ")
plt.ylabel("Relative humidity")
plt.subplot(2,2,4)
plt.scatter(x=forest_fires["rain"],y=forest_fires["DMC"])
plt.xlabel(" rain  value in mm  ")
plt.ylabel("DMC rating value")

Nếu nhiệt độ khoảng 15-20 và giá trị DMC khoảng 120-150 thì đó là điều kiện cực kỳ có khả năng xảy ra cháy rừng có thể gây thiệt hại cho khu vực,

Hầu hết giá trị DMC được tô khi không có mưa

Nếu RH khoảng 40-60 thì giá trị DMC chủ yếu ở khoảng 120-150

Nếu gió khoảng 3-4 km / h và giá trị DMC lớn hơn ở 120-150

<b>DC bị ảnh hưởng như thế nào bởi Nhiệt độ, mưa, gió và RH </b>


In [83]:
plt.figure(figsize=(10,6))
plt.subplot(2,2,1)
plt.scatter(x=forest_fires["temp"],y=forest_fires["DC"])
plt.xlabel(" temp in celcius  ")
plt.ylabel("DC rating value")
plt.subplot(2,2,2)
plt.scatter(x=forest_fires["wind"],y=forest_fires["DC"])
plt.xlabel("wind speed   ")
plt.ylabel("DC rating value ")
plt.subplot(2,2,3)
plt.scatter(x=forest_fires["RH"],y=forest_fires["DC"])
plt.xlabel(" Relative humidity  ")
plt.ylabel("DC rating value")
plt.subplot(2,2,4)
plt.scatter(x=forest_fires["rain"],y=forest_fires["DC"])
plt.xlabel(" rain  value in mm  ")
plt.ylabel("DC rating value")

Nếu nhiệt độ khoảng 15-20 và giá trị DC khoảng 600-800 thì đó là điều kiện cực kỳ có khả năng xảy ra cháy rừng có thể gây thiệt hại cho khu vực

Hầu hết giá trị DC được nhập khi không có mưa

Nếu RH vào khoảng 40-60 thì giá trị DC chủ yếu ở khoảng 600-800

Nếu gió khoảng 3-4 km / h và giá trị DC lớn hơn ở 600-800

<h2>Khu vực bị ảnh hưởng bởi các yếu tố nhiệt độ/ gió/ RH/ mưa </h2>

In [85]:
plt.figure(figsize=(10,8))
plt.subplot(2,2,1)
plt.scatter(x=forest_fires["temp"],y=forest_fires["area"])
plt.xlabel(" temp in celcius  ")
plt.ylabel("area in hectares")
plt.subplot(2,2,2)
plt.scatter(x=forest_fires["wind"],y=forest_fires["area"])
plt.xlabel(" wind speed in km/h ")
plt.ylabel("area in hectares ")
plt.subplot(2,2,3)
plt.scatter(x=forest_fires["RH"],y=forest_fires["area"])
plt.xlabel(" Relative humidity in %  ")
plt.ylabel("area in hectares")
plt.subplot(2,2,4)
plt.scatter(x=forest_fires["rain"],y=forest_fires["area"])
plt.xlabel(" rain in mm")
plt.ylabel("area in hectares ")

Nhiệt độ 15-25 độ ở celcius có khả năng xảy ra cháy rừng cao nhất

Mưa không phải là yếu tố nổi bật

Dưới 60 RH% có thể dẫn đến cháy rừng

Với tốc độ gió 3-4 km / giờ có thể dẫn đến cháy rừng

VẬY, nếu nhiệt độ cao, độ ẩm thấp và không có mưa thì có thể dẫn đến tình trạng cháy rừng

In [86]:
forest_fires.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12), inplace=True)
forest_fires.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7), inplace=True)

In [87]:
forest_fires.describe().T

Nếu area > 0 thì set giá trị = 1 và thay đổi tên cột từ area sang label để có thể coi đó là vấn đề phân loại

In [88]:
forest_fires['area'].values[forest_fires['area'].values > 0] = 1
forest_fires = forest_fires.rename(columns={'area': 'label'})
forest_fires

In [89]:
forest_fires.corr()

In [90]:
forest_fires.corr()['label'].sort_values(ascending=False)

In [91]:
# Gaussian Naive Bayes
from sklearn import metrics
from sklearn.model_selection import train_test_split

X = forest_fires.drop('label', axis=1)
y = forest_fires['label']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=500)

# mô hình Naive Bayes với dữ liệu
G_NB = GaussianNB()
G_NB.fit(X_train,y_train)
print(G_NB)
# đưa ra dự đoán

predict = G_NB.predict(X_test)
# tóm tắt sự phù hợp của mô hình
print(metrics.classification_report(y_test, predict))
print(metrics.confusion_matrix(y_test, predict))

print("Accuracy:",metrics.accuracy_score(y_test, predict))
print("Precision:",metrics.precision_score(y_test, predict))
print("Recall:",metrics.recall_score(y_test, predict))



In [92]:
classes={0:'Safe',1:'On Fire'}
x_new=[[1, 4, 9 ,1 ,91.5, 130.1, 807.1, 7.5, 21.3, 35, 2.2, 0]]
y_predict=G_NB.predict(x_new)
print(classes[y_predict[0]])